In [1]:
import os

from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect
from IPython.display import Markdown, display
from dotenv import load_dotenv
import pandas as pd
import tqdm
import numpy as np

In [2]:
load_dotenv()  # take environment variables from .env.
host="localhost"
database=os.getenv("POSTGRES_DB")
user=os.getenv("POSTGRES_USER")
password=os.getenv("POSTGRES_PASSWORD")
port=os.getenv("POSTGRES_PORT")

db_url = 'postgresql+psycopg2://{user}:{password}@{hostname}:{port}/{database_name}'.format(hostname=host, user=user, password=password, database_name=database, port=5432)

In [3]:
from pathlib import Path

path_raw_data_dir = Path("../../Volumes/data/raw")

files = list(path_raw_data_dir.glob("*.csv"))

In [4]:
# Check all expected 4 files are there
expected_files = {"caracteristiques", "lieux", "usagers", "vehicules"}
for exp_file in expected_files:
    if any(exp_file in f.name for f in files):
        continue
    print(f"Error: could not find file '{exp_file}' in files.")

In [5]:
# check year is the same
fs = [f.name.split(".csv")[0] for f in files]
years = {f.split("-")[-1] for f in fs}
if len(years) > 1:
    print("Error: More than 1 year found in the filenames: {years}")
year = years.pop()
year

'2021'

In [6]:
caracteristiques_file = None
lieux_file = None
usagers_file = None
vehicules_file = None

for file in files:
    if "caracteristiques" in file.name:
        caracteristiques_file = file
    if "lieux" in file.name:
        lieux_file = file
    if "usagers" in file.name:
        usagers_file = file
    if "vehicules" in file.name:
        vehicules_file = file

print(caracteristiques_file, lieux_file, usagers_file, vehicules_file)

../../Volumes/data/raw/caracteristiques-2021.csv ../../Volumes/data/raw/lieux-2021.csv ../../Volumes/data/raw/usagers-2021.csv ../../Volumes/data/raw/vehicules-2021.csv


In [27]:
YEAR = year

#--Importing dataset
df_users = pd.read_csv(usagers_file, sep=";").replace({np.NaN: None})
df_caract = pd.read_csv(caracteristiques_file, sep=";", header=0, low_memory=False).replace({np.NaN: None})
df_places = pd.read_csv(lieux_file, sep = ";", encoding='utf-8')
df_veh = pd.read_csv(vehicules_file, sep=";")

In [28]:
def convert_object_cols_to_str(df):
    stringcols = df.select_dtypes(include='object').columns
    df[stringcols] = df[stringcols].fillna('').astype(str)
    return df

In [29]:
df_places = convert_object_cols_to_str(df_places)

In [8]:
df_users.head()

,Num_Acc,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,secu1,secu2,secu3,locp,actp,etatp
0,202100000001,201Â 764,B01,1,1,3,1,2000.0,1,0,9,-1,0,0,-1
1,202100000001,201Â 765,A01,1,1,1,1,1978.0,1,1,-1,-1,0,0,-1
2,202100000002,201Â 762,A01,1,1,4,1,1983.0,0,1,-1,-1,0,0,-1
3,202100000002,201Â 763,B01,1,1,3,1,1993.0,0,1,-1,-1,0,0,-1
4,202100000003,201Â 761,A01,1,1,1,1,1995.0,1,1,0,-1,0,0,-1


In [9]:
from sqlalchemy import Table, Column, Integer, String, MetaData, create_engine, inspect

engine = create_engine(db_url, echo=False)
inspector = inspect(engine)

In [10]:

from src.data.db.models_sqlalchemy import Base, Caracteristiques, Lieux

In [11]:
# Create table based on the ORM model
Base.metadata.create_all(engine)

In [ ]:
data = (Lieux(**{**row.to_dict(), **{"year":YEAR}}) for i, row in df_places.iterrows())
next(data)

In [12]:
from sqlalchemy.orm import Session

data = (Caracteristiques(**{**row.to_dict(), **{"year":YEAR}}) for i, row in df_caract.iterrows())
with Session(engine) as session:
    session.add_all(data)
    session.commit()

In [18]:
df_places["voie"] = df_places["voie"].astype(str)

In [21]:
df_places.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56518 entries, 0 to 56517
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Num_Acc  56518 non-null  int64 
 1   catr     56518 non-null  int64 
 2   voie     56518 non-null  object
 3   v1       56518 non-null  int64 
 4   v2       4990 non-null   object
 5   circ     56518 non-null  int64 
 6   nbv      56518 non-null  int64 
 7   vosp     56518 non-null  int64 
 8   prof     56518 non-null  int64 
 9   pr       56518 non-null  object
 10  pr1      56518 non-null  object
 11  plan     56518 non-null  int64 
 12  lartpc   108 non-null    object
 13  larrout  56518 non-null  object
 14  surf     56518 non-null  int64 
 15  infra    56518 non-null  int64 
 16  situ     56518 non-null  int64 
 17  vma      56518 non-null  int64 
dtypes: int64(12), object(6)
memory usage: 7.8+ MB


In [24]:
stringcols = df_places.select_dtypes(include='object').columns
stringcols
df_places[stringcols] = df_places[stringcols].fillna('').astype(str)

In [25]:
data = (Lieux(**{**row.to_dict(), **{"year":YEAR}}) for i, row in df_places.iterrows())
with Session(engine) as session:
    session.add_all(data)
    session.commit()

# SQLMODEL

In [30]:
from src.data.db.models import Caracteristiques, Lieux, Vehicules, Users

/Users/evan/Documents/Courses/Datascientest/FinalProject/git/may24_bmlops_accidents/src/data/db/models.py:28: SAWarning: Can't validate argument 'foreign_key'; can't locate any SQLAlchemy dialect named 'foreign'
  Num_Acc: int = Field(default=None, sa_column=Column(BigInteger(), foreign_key="Caracteristiques.Num_Acc"))
/Users/evan/Documents/Courses/Datascientest/FinalProject/git/may24_bmlops_accidents/src/data/db/models.py:50: SAWarning: Can't validate argument 'foreign_key'; can't locate any SQLAlchemy dialect named 'foreign'
  Num_Acc: int = Field(default=None, sa_column=Column(BigInteger(), foreign_key="Caracteristiques.Num_Acc"))
/Users/evan/Documents/Courses/Datascientest/FinalProject/git/may24_bmlops_accidents/src/data/db/models.py:64: SAWarning: Can't validate argument 'foreign_key'; can't locate any SQLAlchemy dialect named 'foreign'
  Num_Acc: int = Field(default=None, sa_column=Column(BigInteger(), foreign_key="Caracteristiques.Num_Acc"))
/Users/evan/Documents/Courses/Datasci

In [31]:
from sqlmodel import SQLModel

SQLModel.metadata.create_all(engine)

In [32]:
with Session(engine) as session:
    for _, row in df_caract.iterrows():
        carac = Caracteristiques(**{**row,**{"year": year}})
        session.add(carac)
    session.commit()

In [33]:
with Session(engine) as session:
    for _, row in df_places.iterrows():
        carac = Lieux(**{**row,**{"year": year}})
        session.add(carac)
    session.commit()

In [ ]:
# compare table data with dataframes
from sqlmodel import select, Session

engine = create_engine(db_url)

with Session(engine) as session:
    statement = select(Caracteristiques).where(Caracteristiques.year==YEAR)
    caracteristiques_from_db = session.exec(statement)

    df= pd.DataFrame([r.model_dump(exclude="year") for r in caracteristiques_from_db.fetchall()])

df.head()


In [ ]:
from sqlalchemy import create_engine

raw_sql_query = "SELECT * FROM {table} WHERE year = {year}"

cnx = create_engine(db_url).connect()
df_caract = pd.read_sql_query(raw_sql_query.format(table=Caracteristiques.__tablename__,year=YEAR), con=cnx)
df_places= pd.read_sql_query(raw_sql_query.format(table=Lieux.__tablename__,year=YEAR), con=cnx)
df_users= pd.read_sql_query(raw_sql_query.format(table=Users.__tablename__,year=YEAR), con=cnx)
df_veh= pd.read_sql_query(raw_sql_query.format(table=Vehicules.__tablename__,year=YEAR), con=cnx)

In [ ]:
from sqlmodel import select

with Session(engine) as session:
    statement = select(Caracteristiques).where(Caracteristiques.year==YEAR)
    caracteristiques_from_db = session.exec(statement)
    df_caract= pd.DataFrame([r.model_dump(exclude="year") for r in caracteristiques_from_db.fetchall()])


In [ ]:
from sqlmodel import select

with Session(engine) as session:
    statement = select(Users, Caracteristiques).where(Users.Num_Acc == Caracteristiques.Num_Acc).limit(10)
    results = session.exec(statement)
    for user, carac in results:
        print("User:", user)
        print("Caracteristiques:", carac)